<img src="hs.png" width="200">

# Sintetički zaposlenici fakulteta #

U ovom notebooku ćemo pokazati kako napraviti lažnu listu zaposlenika nekog fakulteta. 

Na kraju ovog zadatka je proizvedeni JSON file takav da on predstavlja listu zaposlenika. Svaki član liste je dictionary koji ima ključeve:

* ime
* radnoMjesto
* plaća
* kolegiji

Ovaj JSON mora zadovoljavati neka ograničenja:

1. postoje 6 vrsta zaposlenika: dekan, redProf, izvProf, doc, asist, i admin
2. oko 30% zaposlenika je administracija (za naše potrebe, dekan nema nastavne obveze i ubrajamo ga u administraciju)
3. postoji jedan dekan i dekan ima najvišu plaću, koja je 5% veća od sljedeće najviše plaće
4. distribucija znanstveno-nastavnih i suradničkih zvanja (nazivat cemo ih "nastavnici") je uniformna
5. plaće su: redProf= 12000 +/- 10%, izvProf= 10000 +/- 10%, doc= 8000 +/- 10%, asist= 6000 +/- 10%, admin= 8000 +/- 30% . Varijacije su uniformne (odnosno koristite **randint**)
6. broj kolegija je načelno neograničen, njihova imena su kolegij1,kolegij2, kolegij3... U JSONu, svaki nastavnik ima listu svojih **četiri** kolegija. Administracija i dekan imaju praznu listu kolegija.
7. ukupan broj zaposlenika je 150


Prvi korak je da napravimo funkciju koja će generitati plaću oko odredene "osnove" (probajte nekoliko puta pokrenuti sljedeću ćeliju):


In [ ]:
from random import randint
def izrcPlace(osnova, varijacija=0.1):
    return randint(osnova-(osnova*varijacija),osnova+(osnova*varijacija))

print(izrcPlace(10000))
print(izrcPlace(10000, 0.5))

Sljedeće ćemo inicijalizirati vrijednosti i liste koje nam trebaju i koje ćemo onda spajati:

In [ ]:
brojZaposlenika = 150
listaImena = ["osoba"+str(x) for x in range(1,brojZaposlenika+1)]

listaAdmin = ["admin" for x in range(1, (round(brojZaposlenika/3)))]
print(len(listaAdmin)) # neka ostane 49 radi dekana

nastavnici = ["redProf","izvProf","doc","asist"]
brojNastavnika = brojZaposlenika-len(listaAdmin)-1

listaRedProf = ["redProf" for x in range(1,round(brojNastavnika/4)+1)]
listaIzvProf = ["izvProf" for x in range(1,round(brojNastavnika/4)+1)]
listaDoc = ["doc" for x in range(1,round(brojNastavnika/4)+1)]
listaAsist = ["asist" for x in range(1,round(brojNastavnika/4)+1)]
print(len(listaRedProf), len(listaIzvProf), len(listaDoc), len(listaAsist))

Sada možemo početi sastavljati JSON. Sastavit ćemo dvije liste, jednu za nastavnike, a drugu za administraciju i dekana. Prvo krećemo s nastavnicima, jer će nam trebati najveća plaća da izračunamo plaću dekana. Kada je prva verzija gotova, tada ćemo dodati kolegije. Za sada svi dobivaju praznu listu za kolegije. 

Prvo nam treba dovoljno kolegija, sigurno nam je dovoljno njih 1000:

In [ ]:
listaKolegija = ["kolegij"+str(x) for x in range(1,1001)]

Sada radimo prvi dio JSONa, sa nastavnicima:

In [ ]:
from random import shuffle
gotovo_Red_Izv_Doc = []
listaZnNastavnici = listaRedProf + listaIzvProf + listaDoc + listaAsist
shuffle(listaZnNastavnici)
#print(listaZnNastavnici)
counter=0
for i in listaZnNastavnici:
    #print(i)
    novi={}
    novi["radnoMjesto"]=i
    if i=="redProf":
        mark=12000
    elif i=="izvProf":
        mark=10000
    elif i=="doc":
        mark=8000
    elif i=="asist":
        mark=6000
    novi["placa"]= izrcPlace(mark)
    novi["kolegiji"]=[listaKolegija[counter],listaKolegija[counter+1],listaKolegija[counter+2],listaKolegija[counter+3]]
    counter=counter+4
    gotovo_Red_Izv_Doc.append(novi)
print(gotovo_Red_Izv_Doc)
    

Sada trebamo napraviti isto za administraciju (dekana ćemo ručno dodati u sljedećoj ćeliji):

In [ ]:
from random import shuffle
gotovo_Admin = []
shuffle(listaAdmin)
print(len(listaAdmin))
for i in listaAdmin:
    #print(i)
    novi={}
    novi["radnoMjesto"]=i
    mark=8000    
    novi["placa"]= izrcPlace(mark,0.3)
    novi["kolegiji"]=[]
    gotovo_Admin.append(novi)
print(gotovo_Admin)

Sada ručno dodajemo dekana, ali pazimo na izračun plaće:

In [ ]:
skoroGotovo=gotovo_Red_Izv_Doc+gotovo_Admin
place=[x["placa"] for x in skoroGotovo]
najvecaPlaca=max(place)
print(najvecaPlaca)
placaDekana=najvecaPlaca+0.05*najvecaPlaca
dekan=[{"radnoMjesto":"dekan", "placa":placaDekana, "kolegiji":[]}]
print(dekan)
gotovo=skoroGotovo+dekan

Sada spajamo sve u jednu listu, i dopunjujemo s imenima

In [ ]:
counter=0
for i in gotovo:
    i["ime"]=listaImena[counter]
    counter=counter+1
print(gotovo[:5])    

I na kraju, cijelu listu spremamo u JSON file:

In [ ]:
import json
Datoteka=open("faukltet.json","w")
sadrzaj=str(json.dumps(gotovo))
print(sadrzaj)
Datoteka.writelines(sadrzaj)
Datoteka.close()